In [1]:
import pandas as pd
import numpy as np
import bokeh
from scipy.stats import gaussian_kde 
from bokeh.palettes import Blues9
from bokeh.plotting import figure, show, output_file
from bokeh.io import output_notebook, show
from bokeh.sampledata.autompg import autompg as df
from math import sin
from random import random
from bokeh.models import ColumnDataSource, HoverTool, LinearColorMapper, Range1d, CustomJS, TextInput, TabPanel, Tabs, CategoricalColorMapper, WMTSTileSource
from bokeh.palettes import plasma, d3
from bokeh.transform import transform


output_notebook()

import bokeh.sampledata
bokeh.sampledata.download()



Loading BokehJS ...

Using data directory: /Users/masontaylor/.bokeh/data
Skipping 'CGM.csv' (checksum match)
Skipping 'US_Counties.zip' (checksum match)
Skipping 'us_cities.json' (checksum match)
Skipping 'unemployment09.csv' (checksum match)
Skipping 'AAPL.csv' (checksum match)
Skipping 'FB.csv' (checksum match)
Skipping 'GOOG.csv' (checksum match)
Skipping 'IBM.csv' (checksum match)
Skipping 'MSFT.csv' (checksum match)
Skipping 'WPP2012_SA_DB03_POPULATION_QUINQUENNIAL.zip' (checksum match)
Skipping 'gapminder_fertility.csv' (checksum match)
Skipping 'gapminder_population.csv' (checksum match)
Skipping 'gapminder_life_expectancy.csv' (checksum match)
Skipping 'gapminder_regions.csv' (checksum match)
Skipping 'world_cities.zip' (checksum match)
Skipping 'airports.json' (checksum match)
Skipping 'movies.db.zip' (checksum match)
Skipping 'airports.csv' (checksum match)
Skipping 'routes.csv' (checksum match)
Skipping 'haarcascade_frontalface_default.xml' (checksum match)
Skipping 'SampleSuperstore.csv.zip' (

In [36]:
unis = pd.read_csv('../data/IPEDS/university_info/hd2020.csv', encoding='latin-1')
trends = pd.read_csv('../out/trends_black.csv')
merged = pd.merge(unis[['INSTNM', 'UNITID', 'INSTSIZE', 'LONGITUD', 'LATITUDE', 'OBEREG']], trends, on='UNITID')
merged['quadrant']=merged['quadrant'].astype(int)
#color_mapping = {1:'dark_blue', 2:'blue', 3:'dark_red', 4:'red'}
color_mapping = {1:'blue', 2:'lightblue', 3:'red', 4:'pink'}
quadrant_merged = merged
quadrant_merged['color'] = quadrant_merged['quadrant'].replace(color_mapping)
quadrant_merged['INSTSIZE'] = quadrant_merged['INSTSIZE'] * 5
region_merged = merged.copy()
region_mapping = {0: 'black', 1: 'brown', 2: 'pink', 3: 'grey', 4: 'red', 5:'blue', 6:'green', 7:'purple', 8:'orange', 9:'black', -3:'black'}
region_merged['color'] = region_merged['OBEREG'].replace(region_mapping)


In [37]:
import pandas as pd
import numpy as np
def wgs84_to_web_mercator(df, lon="LONGITUD", lat="LATITUDE"):
    """Converts decimal longitude/latitude to Web Mercator format"""
    k = 6378137
    df["x"] = df[lon] * (k * np.pi/180.0)
    df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
    return df

df = wgs84_to_web_mercator(quadrant_merged, lon='LONGITUD', lat='LATITUDE')
hover = HoverTool(tooltips=[
        #("index", "$index"),
        ('Name', '@desc'),
        ("Non-CS stat", "@noncs_stat"),
        ("CS stat", "@cs_stat"),
        ("Size", "@size")
    ])
data={'x_values':df['x'].tolist(), 'y_values':df['y'].tolist(), 'desc':df['INSTNM'].tolist(), \
    'color':df['color'].tolist(), 'size':df['INSTSIZE'].tolist(), 'cs_stat':df['cs_stat'].tolist(),\
        'noncs_stat':df['noncs_stat'].tolist(), 'radius':df['INSTSIZE'].tolist()}
source = ColumnDataSource(data=data)
USA = x_range,y_range = ((-13884029,-7453304), (2698291,6455972))
url = 'http://a.basemaps.cartocdn.com/rastertiles/voyager/{Z}/{X}/{Y}.png'
attribution = "Tiles by Carto, under CC BY 3.0. Data by OSM, under ODbL"
p = figure(sizing_mode='stretch_both', tools=['pan, wheel_zoom', hover, 'save'], x_range=x_range, y_range=y_range, 
           x_axis_type="mercator", y_axis_type="mercator")

p.add_tile(WMTSTileSource(url=url, attribution=attribution))
p.add_tile("CartoDB Positron", retina=True)

p.circle(x='x_values', y='y_values', size='size', fill_color='color', name='desc', alpha=0.6, source=source)
#mapper = LinearColorMapper(palette=plasma(256), low=min(list_y), high=max(list_y))

#output_file('quadrant_map.html')
show(p)

In [4]:
# import pandas as pd
# import numpy as np

# def wgs84_to_web_mercator(df, lon="lon", lat="lat"):
#     """Converts decimal longitude/latitude to Web Mercator format"""
#     k = 6378137
#     df["x"] = df[lon] * (k * np.pi/180.0)
#     df["y"] = np.log(np.tan((90 + df[lat]) * np.pi/360.0)) * k
#     return df

# #df = pd.DataFrame(dict(name=["Austin", "NYC"], lon=[-97.7431,-74.0059], lat=[30.2672,40.7128]))
# df = wgs84_to_web_mercator(region_merged, lon='LONGITUD', lat='LATITUDE')
# hover = HoverTool(tooltips=[
#         # ("index", "$index"),
#         ('Name', '@INSTNM'),
#         ("Non-CS stat", "@x"),
#         ("CS stat", "@y"),
#         ("Size", "@INSTSIZE")
#     ])
# p = figure(tools=['pan, wheel_zoom', hover], x_range=x_range, y_range=y_range, 
#            x_axis_type="mercator", y_axis_type="mercator")

# p.add_tile(WMTSTileSource(url=url, attribution=attribution))

# source= ColumnDataSource(df)
# palette = d3['Category10'][len(df['quadrant'].unique())]

# #color_map=CategoricalColorMapper(factors=df['quadrant'].unique(), palette=palette)
# #p.scatter(x='LONGITUD', y='LATITUDE', color={'field': 'quadrant', 'transform': color_map}, source=source)
# p.circle(x=df['x'], y=df['y'], size=5, alpha=0.7, fill_color=df['color'], line_color='black')
# #p.circle(x=df['x'], y=df['y'], size=(df['INSTSIZE'] * 2), alpha=0.6, fill_color='blue')
# output_file('region_map.html')
# show(p)